In [1]:
import numpy as np
import pandas as pd

import re
import string

import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from pymongo import MongoClient

### Load data from MongoDB in pandas dataframe

In [2]:
client = MongoClient()
client.list_database_names()

['admin', 'jobs_data', 'local']

In [3]:
db = client.jobs_data
db.list_collection_names()

['Chicago_Jobs',
 'Los_Angeles_Jobs',
 'SF_Bay_Area_Jobs',
 'Seattle_Jobs',
 'New_York_Jobs']

In [4]:
cursor = db.SF_Bay_Area_Jobs.find({}, 
    {'_id':0, 'url':1, 'job_title':1, 'company_name':1, 'listed_items':1})
df = pd.DataFrame(list(cursor))
df = df.drop_duplicates('listed_items')
df.head()

,company_name,job_title,listed_items,url
0,24 Hour Fitness,Real Estate/Financial Analyst,Financial Analysis: 3 years (Preferred),https://www.indeed.com/company/24-Hour-Fitness...
1,Affimedix Inc,Scientist - Molecular Biology,Molecular cloning of DNA libraries. Library sc...,https://www.indeed.com/company/Affimedix-Inc/j...
2,"AllAccem, Inc.","Associate Chemist, Production Chemistry",Assist in carrying out SOP chemical reactions ...,"https://www.indeed.com/company/AllAccem,-Inc./..."
3,Alveo Technologies,Scientist I,Perform experiments at the bench and on protot...,https://www.indeed.com/company/Alveo-Technolog...
5,Alveo Technologies,Senior Scientist,Work closely with the Director of Assay Develo...,https://www.indeed.com/company/Alveo-Technolog...


### Remove non-word characters and make lower
Note that "listed_items" refers to anything that was bulleted/listed in the job description. Paragraph form text was not used.

In [5]:
alphabet = lambda x: re.sub('[\W]+|[0-9]+', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
double_space = lambda X: re.sub(' +', ' ',string4)

df['listed_items'] = df.listed_items.map(alphabet).map(punc_lower)
df.head()

,company_name,job_title,listed_items,url
0,24 Hour Fitness,Real Estate/Financial Analyst,financial analysis years preferred,https://www.indeed.com/company/24-Hour-Fitness...
1,Affimedix Inc,Scientist - Molecular Biology,molecular cloning of dna libraries library scr...,https://www.indeed.com/company/Affimedix-Inc/j...
2,"AllAccem, Inc.","Associate Chemist, Production Chemistry",assist in carrying out sop chemical reactions ...,"https://www.indeed.com/company/AllAccem,-Inc./..."
3,Alveo Technologies,Scientist I,perform experiments at the bench and on protot...,https://www.indeed.com/company/Alveo-Technolog...
5,Alveo Technologies,Senior Scientist,work closely with the director of assay develo...,https://www.indeed.com/company/Alveo-Technolog...


Some listed_items fields are blank so we'll just get rid of them.

In [6]:
blank_indeces = df[df.listed_items == ''].index
df = df.drop(blank_indeces)

### Remove common english stop words

In [7]:
stop = set(stopwords.words('english'))

In [8]:
df['listed_items'] = df.listed_items.apply(lambda x: 
                [item for item in x.split() if item not in stop])
df['listed_items'] = df.listed_items.str.join(' ')
df.head()

,company_name,job_title,listed_items,url
0,24 Hour Fitness,Real Estate/Financial Analyst,financial analysis years preferred,https://www.indeed.com/company/24-Hour-Fitness...
1,Affimedix Inc,Scientist - Molecular Biology,molecular cloning dna libraries library screen...,https://www.indeed.com/company/Affimedix-Inc/j...
2,"AllAccem, Inc.","Associate Chemist, Production Chemistry",assist carrying sop chemical reactions gram mu...,"https://www.indeed.com/company/AllAccem,-Inc./..."
3,Alveo Technologies,Scientist I,perform experiments bench prototype devices su...,https://www.indeed.com/company/Alveo-Technolog...
5,Alveo Technologies,Senior Scientist,work closely director assay development achiev...,https://www.indeed.com/company/Alveo-Technolog...


### Stemming and Lemmatization
Lemmatization was chosen over stemming for the sake of interpreting ngrams.  
I felt comfortable doing this despite how expensive lemmatization is compared to stemming due to the fact that my dataset was not particularly large.

In [11]:
lem = WordNetLemmatizer()

In [12]:
def description_lemmatizer(description):
    tokens = description.split()
    stemmed_tokens = [lem.lemmatize(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [13]:
df['listed_items'] = df.listed_items.apply(description_lemmatizer)
df.head()

,company_name,job_title,listed_items,url
0,24 Hour Fitness,Real Estate/Financial Analyst,financial analysis year preferred,https://www.indeed.com/company/24-Hour-Fitness...
1,Affimedix Inc,Scientist - Molecular Biology,molecular cloning dna library library screenin...,https://www.indeed.com/company/Affimedix-Inc/j...
2,"AllAccem, Inc.","Associate Chemist, Production Chemistry",assist carrying sop chemical reaction gram mul...,"https://www.indeed.com/company/AllAccem,-Inc./..."
3,Alveo Technologies,Scientist I,perform experiment bench prototype device supp...,https://www.indeed.com/company/Alveo-Technolog...
5,Alveo Technologies,Senior Scientist,work closely director assay development achiev...,https://www.indeed.com/company/Alveo-Technolog...


The below were added much later on after KMeans clustering and NMF topic modeling had been performed. You'll notice that the middle section is composed mostly of 'perk' terms and actually had their own topic when doing topic modeling with both LDA and NMF.

In [15]:
nmf_tfidf_stopwords = ['market', 'roadmap', 'experience', 'need',
 'solution', 'technology', 'requirement', 'ability', 'user',
 'new', 'science', 'scientist', 'service', 'organization', 'working',
 'year', 'uber', 'system', 'software', 'technology', 'year', 'service',
 'computer', 'working', 'etc', 'equivalent', 'modern', 'degree',
 'feature', 'good', 'skill', 'google', 'sample', 'including', 'ng',
 'benefit', 'dental', 'paid', 'vision', 'medical', 'lunch', 'employee',
 'insurance', 'medical_dental', 'commuter', 'company', 'health',
 'flexible', 'snack', 'discount', '401k', 'catered', 'commuter_benefit',
 'leave', 'vacation', 'competitive', 'plan', 'holiday', 'equity',
 'time', 'coverage', 'salary', 'free', 'office', 'gym', 'drink',
 'program', 'fully', 'generous', 'parental', 'competitive_salary',
 '401', 'day', 'catered_lunch', '100', 'wellness', 'dependent', 'week',
 'daily', 'stipend', 'stocked', 'pto', 'reimbursement', 'membership',
 'family', 'matching', 'monthly', 'disability', 'help', 'kitchen',
 'world', 'francisco', 'bonus', 'life', 'opportunity', 'career', 'san',
 'hour', 'people', 'open', 'unlimited', 'every', 'tax', 'location',
 'phone', 'term', 'volunteer', 'juul', 'subsidy', 'great', 'site',
 'breakfast', 'policy', 'dinner', 'parking', 'talented', 'growth',
 'option', 'compensation', 'experience', 'grow', 'stock', 'supportive',
 'contribution', 'place', 'maternity', 'pre', 'one', 'offer', 'premium',
 'growing', 'match', 'annual', 'fun', 'culture', 'happy', 'retirement',
 'long', 'care', 'healthy', 'heart', 'fsa', 'committed', 'coffee',
 'take', 'environment', 'goal', 'teammate', 'transportation', 'spending',
 'food', 'dress', 'exceed', 'short', 'boundless', 'package', 'full', 
 'sponsored', 'get', 'tech', 'twitch', 'plus', 'schedule', 'event', 
 'meaningful', 'massage', 'greatest', 'comprehensive', 'choice', 'brand', 
 'year', 'downtown', 'excellent', 'real', 'healthcare', 'per', 'perk', 
 'talk', 'fast', 'success', 'consumer', 'pay', 'top', 'big', 'assistance', 
 'engineering', 'casual', 'home', 'professional', 'best', 'view', 
 'served', 'performance', 'set', 'tuition', 'class', 'friday', '10', 
 'always', 'regular', 'amazon', 'smart', 'give', 'want', 'including', 
 'meet', 'month', 'tap', 'rate', 'employer', 'station', 'talent', 'space', 
 'high', 'monday', 'non', 'bi', 'skill', 'must', 'manufacturing', 'able',
 'sexual', 'orientation',  'gender',  'identity', 'www', 'linkedin', 'com',
 'data', 'code', 'team', 'work']

In [17]:
df['listed_items'] = df.listed_items.apply(lambda x: 
                [item for item in x.split() if item not in nmf_tfidf_stopwords])
df['listed_items'] = df.listed_items.str.join(' ')
df.head()

,company_name,job_title,listed_items,url
0,24 Hour Fitness,Real Estate/Financial Analyst,financial analysis preferred,https://www.indeed.com/company/24-Hour-Fitness...
1,Affimedix Inc,Scientist - Molecular Biology,molecular cloning dna library library screenin...,https://www.indeed.com/company/Affimedix-Inc/j...
2,"AllAccem, Inc.","Associate Chemist, Production Chemistry",assist carrying sop chemical reaction gram mul...,"https://www.indeed.com/company/AllAccem,-Inc./..."
3,Alveo Technologies,Scientist I,perform experiment bench prototype device supp...,https://www.indeed.com/company/Alveo-Technolog...
5,Alveo Technologies,Senior Scientist,closely director assay development achieve pro...,https://www.indeed.com/company/Alveo-Technolog...


In [19]:
df['string_len'] = df.listed_items.apply(lambda x: len(x.split()))
len(df)

3811

In [21]:
# Removes descriptions with only a few words
df = df[df.string_len > 20]
len(df)

3760

In [24]:
df.drop('string_len', axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)
df.to_pickle('../Data/01_clean_sf')